In [6]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split

ValueError: numpy.dtype size changed, may indicate binary incompatibility. Expected 96 from C header, got 88 from PyObject

In [ ]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [ ]:
X_train=pd.read_csv(r"C:\Users\DELL\OneDrive\Desktop\AI & DATA SCIENCE\Sem 2\ML 2\Projects\Project1\X_train.csv")
X_test=pd.read_csv(r"C:\Users\DELL\OneDrive\Desktop\AI & DATA SCIENCE\Sem 2\ML 2\Projects\Project1\X_test.csv")
X_val=pd.read_csv(r"C:\Users\DELL\OneDrive\Desktop\AI & DATA SCIENCE\Sem 2\ML 2\Projects\Project1\X_val.csv")
y_val=pd.read_csv(r"C:\Users\DELL\OneDrive\Desktop\AI & DATA SCIENCE\Sem 2\ML 2\Projects\Project1\y_val.csv")
y_train=pd.read_csv(r"C:\Users\DELL\OneDrive\Desktop\AI & DATA SCIENCE\Sem 2\ML 2\Projects\Project1\y_train.csv")
y_test=pd.read_csv(r"C:\Users\DELL\OneDrive\Desktop\AI & DATA SCIENCE\Sem 2\ML 2\Projects\Project1\y_test.csv")

In [ ]:
X_train.head(1)

In [ ]:
X_train = X_train.drop(['state','city','review_count_y','stars','elite_user_count','total_useful_votes'], axis=1)
X_val = X_val.drop(['state','city','review_count_y','stars','elite_user_count','total_useful_votes'], axis=1)
X_test = X_test.drop(['state','city','review_count_y','stars','elite_user_count','total_useful_votes'], axis=1)

In [ ]:
# Apply pd.get_dummies() to both X_train and X_test
X_train_encoded = pd.get_dummies(X_train)
X_test_encoded = pd.get_dummies(X_test)
X_val_encoded= pd.get_dummies(X_val)
# Align the columns of X_test to match X_train
X_test_encoded = X_test_encoded.reindex(columns=X_train_encoded.columns, fill_value=0)
X_val_encoded = X_val_encoded.reindex(columns=X_train_encoded.columns, fill_value=0)

In [ ]:
y_train.value_counts(normalize=True) * 100

In [ ]:
X_train_encoded.isnull().sum()

In [ ]:
print(X_train_encoded.shape)
print(y_train.shape)

In [ ]:
# Drop rows with null values in X_train_encoded
X_train_encoded_clean = X_train_encoded.dropna()

# Make sure to drop corresponding rows in y_train using the same index
y_train_clean = y_train.loc[X_train_encoded_clean.index]

In [ ]:
print(X_train_encoded_clean.shape)
print(y_train_clean.shape)

In [ ]:
# Apply SMOTE to the training data
smote = SMOTE(random_state=42)
X_train_smote, y_train_smote = smote.fit_resample(X_train_encoded_clean, y_train_clean)

In [ ]:
y_train_smote.value_counts(normalize=True) * 100

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, classification_report

# Train a Decision Tree Classifier
dt_model = DecisionTreeClassifier(
                                     max_depth=10,               # Limit the depth of the tree
    min_samples_split=10,        # Require at least 10 samples to split an internal node
    min_samples_leaf=10,          # Require at least 5 samples at leaf nodes
    max_leaf_nodes=100,           # Limit the number of leaf nodes
    max_features='sqrt',         # Use the square root of the number of features at each split (good for classification tasks)
    random_state=42              # Ensures reproducibility
                                 )
dt_model.fit(X_train_smote, y_train_smote)

In [ ]:
# Make predictions on the training data
y_train_pred = dt_model.predict(X_train_smote)

# Calculate and print training accuracy
training_accuracy = accuracy_score(y_train_smote, y_train_pred)
print(f"Training Accuracy: {training_accuracy:.4f}")

In [ ]:
# Predict on val data
y_pred_dt = dt_model.predict(X_val_encoded)

# Evaluate the model
print("Decision Tree validation Accuracy :", accuracy_score(y_val, y_pred_dt))
print(classification_report(y_val, y_pred_dt))

In [ ]:
# Get feature importance
importance = dt_model.feature_importances_

# Create a DataFrame for feature importance
feature_names = X_train_encoded.columns
importance_df = pd.DataFrame({'Feature': feature_names, 'Importance': importance})
importance_df = importance_df.sort_values(by='Importance', ascending=False).head(20)

# Set the size of the plot
plt.figure(figsize=(12, 8))

# Create a bar plot
sns.barplot(x='Importance', y='Feature', data=importance_df)

# Add titles and labels
plt.title('Top 10 Feature Importance from DecisionTreeClassifier')
plt.xlabel('Importance')
plt.ylabel('Feature')

# Add the feature importance values beside the bars
for index, value in enumerate(importance_df['Importance']):
    plt.text(value, index, f'{value:.3f}', va='center')  # Format to 4 decimal places


# Show the plot
plt.show()

In [ ]:
X_train_smote['RestaurantsPriceRange2']=X_train_smote['RestaurantsPriceRange2'].astype('object')

In [ ]:
dt_model.fit(X_train_smote, y_train_smote)

In [ ]:
# Make predictions on the training data
y_train_pred = dt_model.predict(X_train_smote)

# Calculate and print training accuracy
training_accuracy = accuracy_score(y_train_smote, y_train_pred)
print(f"Training Accuracy: {training_accuracy:.4f}")

In [ ]:
# Predict on val data
y_pred_dt = dt_model.predict(X_val_encoded)

# Evaluate the model
print("Decision Tree Accuracy:", accuracy_score(y_val, y_pred_dt))
print(classification_report(y_val, y_pred_dt))

In [ ]:
# Get feature importance
importance_2 = dt_model.feature_importances_

# Create a DataFrame for feature importance
feature_names = X_train_encoded.columns
importance_df_2 = pd.DataFrame({'Feature': feature_names, 'Importance': importance_2})
importance_df_2 = importance_df_2.sort_values(by='Importance', ascending=False).head(20)

# Set the size of the plot
plt.figure(figsize=(12, 8))

# Create a bar plot
sns.barplot(x='Importance', y='Feature', data=importance_df_2)

# Add titles and labels
plt.title('Top 20 Feature Importance from DecisionTreeClassifier')
plt.xlabel('Importance')
plt.ylabel('Feature')

# Add the feature importance values beside the bars
for index, value in enumerate(importance_df_2['Importance']):
    plt.text(value, index, f'{value:.3f}', va='center')  # Format to 4 decimal places


# Show the plot
plt.show()

In [ ]:
X_train_smote['RestaurantsPriceRange2']=X_train_smote['RestaurantsPriceRange2'].astype('category')

In [ ]:
dt_model.fit(X_train_smote, y_train_smote)

In [ ]:
# Make predictions on the training data
y_train_pred = dt_model.predict(X_train_smote)

# Calculate and print training accuracy
training_accuracy = accuracy_score(y_train_smote, y_train_pred)
print(f"Training Accuracy: {training_accuracy:.4f}")

In [ ]:
# Predict on val data
y_pred_dt = dt_model.predict(X_val_encoded)

# Evaluate the model
print("Decision Tree Accuracy:", accuracy_score(y_val, y_pred_dt))
print(classification_report(y_val, y_pred_dt))

In [ ]:
# Get feature importance
importance_3 = dt_model.feature_importances_

# Create a DataFrame for feature importance
feature_names = X_train_encoded.columns
importance_df_3 = pd.DataFrame({'Feature': feature_names, 'Importance': importance_3})
importance_df_3 = importance_df_3.sort_values(by='Importance', ascending=False).head(20)

# Set the size of the plot
plt.figure(figsize=(12, 8))

# Create a bar plot
sns.barplot(x='Importance', y='Feature', data=importance_df_3)

# Add titles and labels
plt.title('Top 20 Feature Importance from DecisionTreeClassifier')
plt.xlabel('Importance')
plt.ylabel('Feature')

# Add the feature importance values beside the bars
for index, value in enumerate(importance_df_3['Importance']):
    plt.text(value, index, f'{value:.3f}', va='center')  # Format to 4 decimal places


# Show the plot
plt.show()

In [ ]:
from sklearn.model_selection import GridSearchCV

In [ ]:
# Define the parameter grid for tuning
param_grid = {
    'max_depth': [5,10, 15],
    'min_samples_split': [2, 5, 10,20],
    'min_samples_leaf': [1,3,5,7,10],
    'max_leaf_nodes': [20,40,60,80,100],
    'max_features': [None, 'sqrt', 'log2']
}

# Initialize the Decision Tree Classifier
dt_model = DecisionTreeClassifier(random_state=42)

# Initialize Grid Search
grid_search = GridSearchCV(estimator=dt_model, 
                           param_grid=param_grid, 
                           cv=5,  # 5-fold cross-validation
                           scoring='accuracy',  # Use accuracy as the scoring metric
                           n_jobs=-1,  # Use all available cores
                           verbose=2)  # Print progress

# Fit the model using Grid Search
grid_search.fit(X_train_smote, y_train_smote)

# Get the best parameters and the best score
best_params = grid_search.best_params_
best_score = grid_search.best_score_

print("Best Parameters: ", best_params)
print("Best Cross-Validation Score: {:.4f}".format(best_score))


In [ ]:
# Train a Decision Tree Classifier
best_model = DecisionTreeClassifier(**best_params)
best_model.fit(X_train_smote, y_train_smote)

In [ ]:
# Make predictions on the training data
y_train_pred = best_model.predict(X_train_smote)

# Calculate and print training accuracy
training_accuracy = accuracy_score(y_train_smote, y_train_pred)
print(f"Training Accuracy: {training_accuracy:.4f}")

In [ ]:
# Predict on test data
y_pred_dt = best_model.predict(X_test_encoded)

# Evaluate the model
print("Decision Tree Accuracy:", accuracy_score(y_test, y_pred_dt))
print(classification_report(y_test, y_pred_dt))

In [ ]:
# Get feature importance
importance_best_model = best_model.feature_importances_

# Create a DataFrame for feature importance
feature_names = X_train_encoded.columns
importance_df__best_model = pd.DataFrame({'Feature': feature_names, 'Importance': importance_best_model})
importance_df__best_model = importance_df__best_model.sort_values(by='Importance', ascending=False).head(20)

# Set the size of the plot
plt.figure(figsize=(12, 8))

# Create a bar plot
sns.barplot(x='Importance', y='Feature', data=importance_df__best_model)

# Add titles and labels
plt.title('Top 20 Feature Importance from DecisionTreeClassifier')
plt.xlabel('Importance')
plt.ylabel('Feature')

# Add the feature importance values beside the bars
for index, value in enumerate(importance_df__best_model['Importance']):
    plt.text(value, index, f'{value:.3f}', va='center')  # Format to 4 decimal places


# Show the plot
plt.show()

In [ ]:
# Check for negative values
negative_values = (X_train_encoded < 0).sum()
print("Negative values per feature:\n", negative_values)

# Check for zero values
zero_values = (X_train_encoded == 0).sum()
print("Zero values per feature:\n", zero_values)

In [ ]:
from sklearn.preprocessing import StandardScaler

# Scale the data
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_encoded)
X_test_scaled = scaler.transform(X_test_encoded)

# Now use X_train_scaled and X_test_scaled for LIME
explainer = lime.lime_tabular.LimeTabularExplainer(
    X_train_scaled,  # Scaled training data
    feature_names=X_train_encoded.columns,  # Feature names
    class_names=['Class 0', 'Class 1'],  # Adjust based on your problem
    mode='classification'
)

# Pick a sample from the test set to explain
i = 10
sample = X_test_scaled[i].reshape(1, -1)

# Generate explanation
exp = explainer.explain_instance(
    data_row=X_test_scaled[i],  # Scaled instance to explain
    predict_fn=best_model.predict_proba  # Prediction function
)

# Show explanation
exp.show_in_notebook(show_table=True)


In [ ]:
#pip install numba==1.24.0 --force-reinstall 
import shap

# Initialize the SHAP explainer for the Decision Tree model
explainer = shap.TreeExplainer(best_model)

# Compute SHAP values for the test set
shap_values = explainer.shap_values(X_test_encoded)

# Plot SHAP summary plot for global feature importance
shap.summary_plot(shap_values[1], X_test_encoded, feature_names=X_test_encoded.columns)

In [ ]:
import shap

# Initialize the SHAP explainer for the Decision Tree model
explainer = shap.TreeExplainer(best_model)

# Compute SHAP values for the test set
shap_values = explainer.shap_values(X_test_encoded)

# Plot SHAP values for the first prediction (or for all)
shap.initjs()  # Initialize JS visualization in Jupyter Notebook

# Summary plot
shap.summary_plot(shap_values, X_test_encoded, feature_names=X_test_encoded.columns)

# You can also visualize SHAP values for a specific instance
# For example, explaining the prediction for the first test instance
shap.force_plot(explainer.expected_value, shap_values[1][0], X_test_encoded.iloc[0])